<a href="https://colab.research.google.com/github/arunprakashpj/PythonMiniProjects/blob/master/Copy_of_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install apache-beam
import apache_beam as beam

     |████████████████████████████████| 3.4MB 1.3MB/s 
     |████████████████████████████████| 1.2MB 15.8MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 225kB 20.2MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 59.2MB 71kB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 112kB 46.9MB/s 
  Created wheel for hdfs: filename=hdfs-2.5.8-cp36-none-any.whl size=33213 sha256=4b2239fabe6671f87f2dd4dbfd16819f5d8fe18ca5d41cc1aec94a55b6da651a
  Stored in directory: /root/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70
  Created wheel for httplib2: filename=httplib2-0.12.0-cp36-none-any.whl size=93464 sha256=d04af2bd29991a4c650c7a699826ea2ed8da912fdfbf90fbed28965659b81672
  Stored in directory: /root/.cache/pip/wheels/6d/41/4b/2b369d6e2b7eaebcdd423516d3fb659c7658c16a2be8fd04ec
  Created wheel for avro-python3: filename=

In [0]:
from google.colab import files
uploaded = files.upload()

Saving dept_data.txt to dept_data.txt


In [0]:
!ls

dept_data.txt  sample_data


In [0]:
cat dept_data.txt

In [0]:
!{'mkdir -p data'}

In [0]:
!ls

data  dept_data.txt  sample_data


In [0]:
from google.colab import files
uploaded = files.upload()

Saving data.txt to data.txt


In [0]:
import apache_beam as beam

def SplitRow(element):
    return element.split(',')

def filtering(record):
  return record[3] == 'Accounts'


p1 = beam.Pipeline()

attendance_count = (
    
   p1
    |beam.io.ReadFromText('dept_data.txt')
    |beam.Map(SplitRow)
   #| beam.Map(lambda record: record.split(','))

    |beam.Filter(filtering)
  # |beam.Filter(lambda record: record[3] == 'Accounts')
    
    |beam.Map(lambda record: (record[1], 1))
    |beam.CombinePerKey(sum)
    
    |beam.io.WriteToText('data/output_new_final')
  
)

p1.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/output_new_final-00000-of-00001')}

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [0]:
import apache_beam as beam

class SplitRow(beam.DoFn):
  
  def process(self, element):
    # return type -> list
    return  [element.split(',')]
  

class FilterAccountsEmployee(beam.DoFn):
  
  def process(self, element):
    if element[3] == 'Accounts':
      return [element]  
    
class PairEmployees(beam.DoFn):
  
  def process(self, element):
    return [(element[3]+","+element[1], 1)]    
  
class Counting(beam.DoFn):
  
  def process(self, element):
    # return type -> list
    (key, values) = element           # [Marco, Accounts  [1,1,1,1....] , Rebekah, Accounts [1,1,1,1,....] ]
    return [(key, sum(values))]
     

p1 = beam.Pipeline()

attendance_count = (
    
   p1
    |beam.io.ReadFromText('dept_data.txt')
    
    |beam.ParDo(SplitRow())
   # | 'Compute WordLength' >> beam.ParDo(lambda element: [ element.split(',') ]) 

    |beam.ParDo(FilterAccountsEmployee())
    |beam.ParDo(PairEmployees())
    | 'Group ' >> beam.GroupByKey()
    | 'Sum using ParDo' >> beam.ParDo(Counting())  
    
    |beam.io.WriteToText('data/output_new_final')
  
)

p1.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/output_new_final-00000-of-00001')}

('Accounts,Marco', 31)
('Accounts,Rebekah', 31)
('Accounts,Itoe', 31)
('Accounts,Edouard', 31)
('Accounts,Kyle', 62)
('Accounts,Kumiko', 31)
('Accounts,Gaston', 31)
('Accounts,Ayumi', 30)


In [0]:
import apache_beam as beam

p = beam.Pipeline()

class AverageFn(beam.CombineFn):
  
  def create_accumulator(self):
     return (0.0, 0)   # initialize (sum, count)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    
    ind_sums, ind_counts = zip(*accumulators)       # zip - [(27, 3), (39, 3), (18, 2)]  -->   [(27,39,18), (3,3,2)]
    return sum(ind_sums), sum(ind_counts)        # (84,8)

  def extract_output(self, sum_count):    
    
    (sum, count) = sum_count    # combine globally using CombineFn
    return sum / count if count else float('NaN')
  

small_sum = (
           p 
            | beam.Create([15,5,7,7,9,23,13,5])
            | "Combine Globally" >> beam.CombineGlobally(AverageFn()) 
            | 'Write results' >> beam.io.WriteToText('data/combine')
          )
p.run()

# Sample the first 20 results, remember there are no ordering guarantees.
!{'head -n 20 data/combine-00000-of-00001'}

10.5
